# Data files set name

### Dependencies:

In [ ]:
import os

import pandas as pd
import copy
from tabulate import tabulate
import decimal

from library import filesystem_utilities, data_processing

### Set constants:

In [ ]:
from library import ROOT

PROCESSED_PARAMETER_VALUES_CSV_FILE_FULL_PATH = os.path.join(
    ROOT, "data_files/processed/invert/Chebyshev_several_config_varying_N/processed_parameter_values.csv",
)
if not filesystem_utilities.is_valid_file(PROCESSED_PARAMETER_VALUES_CSV_FILE_FULL_PATH):
    raise ValueError("Invalid .csv file path.")


DATA_FILES_SET_PLOTS_DIRECTORY = os.path.join(
    ROOT, "output/plots/invert/Chebyshev_several_config_varying_N",
)
if not filesystem_utilities.is_valid_directory(DATA_FILES_SET_PLOTS_DIRECTORY):
    raise ValueError("Invalid plots base directory path.")

### Import .csv file and initialize objects:

In [ ]:
processed_parameter_values_dataframe = data_processing.load_csv(PROCESSED_PARAMETER_VALUES_CSV_FILE_FULL_PATH)
processed_parameter_values_analyzer = data_processing.DataFrameAnalyzer(processed_parameter_values_dataframe)

### DataFrame fields statistics:

In [ ]:
single_valued_fields_dictionary = processed_parameter_values_analyzer.single_valued_fields_dictionary
multivalued_fields_dictionary = processed_parameter_values_analyzer.multivalued_fields_dictionary
data_processing.print_dictionaries_side_by_side(
    single_valued_fields_dictionary, multivalued_fields_dictionary,
    left_column_title="Single-valued fields : unique value", 
    right_column_title="Multivalued fields : # of unique values")

Single-valued fields : unique value   | Multivalued fields : # of unique values
--------------------------------------------------------------------------------
Main_program_type: invert               Filename: 1333
Threads_per_process: 1                  Kernel_operator_type: 2
QCD_beta_value: 6.20                    MPI_geometry: 4
APE_alpha: 0.72                         Configuration_label: 65
APE_iterations: 1                       Bare_mass: 13
Rho_value: 1.0                          Plaquette: 65
Clover_coefficient: 0                   Number_of_Chebyshev_terms: 42
Lanczos_epsilon: 1e-10                  Minimum_eigenvalue_squared: 113
Maximum_Lanczos_iterations: 10000       Maximum_eigenvalue_squared: 116
Delta_Min: 0.50                         Total_overhead_time: 1333
Delta_Max: 1.10                         Total_number_of_Lanczos_iterations: 84
Number_of_spinors: 12                   Total_calculation_time: 1333
Overlap_operator_method: Chebyshev      Condition_number: 116
CG

## Tables

## Bare_mass Vs Number_of_Chebyshev_terms

In [ ]:
processed_parameter_values_analyzer.dataframe = processed_parameter_values_analyzer.dataframe[
    ((processed_parameter_values_analyzer.dataframe["Number_of_Chebyshev_terms"] >= 15)
    & (processed_parameter_values_analyzer.dataframe["Number_of_Chebyshev_terms"] <= 120)
    & (processed_parameter_values_analyzer.dataframe["Kernel_operator_type"] == 'Brillouin'))
    | ((processed_parameter_values_analyzer.dataframe["Number_of_Chebyshev_terms"] >= 90)
    & (processed_parameter_values_analyzer.dataframe["Number_of_Chebyshev_terms"] <= 300)
    & (processed_parameter_values_analyzer.dataframe["Kernel_operator_type"] == 'Wilson'))
]

# Store names and values of tunable parameters with unique values
single_valued_fields_dictionary = copy.deepcopy(
    processed_parameter_values_analyzer.single_valued_fields_dictionary
)

# CONSTRUCT LIST OF RELEVANT MULTIVALUED TUNABLE PARAMETERS FOR GROUPING

# Groupings will be based on tunable parameters with more than one
# unique values (multivalued)
tunable_multivalued_parameter_names_list = copy.deepcopy(
    processed_parameter_values_analyzer.list_of_tunable_multivalued_parameter_names
)

# Remove specific parameters from the list
parameters_to_remove = ["MPI_geometry", "Configuration_label", "Number_of_Chebyshev_terms", "Bare_mass"]
tunable_multivalued_parameter_names_list = [
    parameter_name
    for parameter_name in tunable_multivalued_parameter_names_list
    if parameter_name not in parameters_to_remove
]

# LOOP OVER ALL RELEVANT TUNABLE PARAMETERS GROUPINGS

# Include counting the iterations for later use
for combination_of_values, dataframe_group in processed_parameter_values_analyzer.dataframe.groupby(tunable_multivalued_parameter_names_list, observed=True):
    # Store specific tunable multivalued parameter names and values in a
    # dedicated metadata dictionary for later use
    if not isinstance(combination_of_values, tuple):
        combination_of_values = [combination_of_values]
    metadata_dictionary = dict(
        zip(tunable_multivalued_parameter_names_list, combination_of_values)
    )

    print(metadata_dictionary)

    table = pd.pivot_table(dataframe_group, 
                       values="Configuration_label", 
                       index="Bare_mass", 
                       columns="Number_of_Chebyshev_terms", 
                    #    aggfunc=lambda x: list(x), 
                    #    fill_value="")
                        aggfunc="count",
                        fill_value=0)

    print(tabulate(table, headers="keys", tablefmt="pipe"))

processed_parameter_values_analyzer.restore_entire_dataframe()

{'Kernel_operator_type': 'Wilson'}
|   Bare_mass |   90 |   100 |   110 |   120 |   130 |   140 |   150 |   160 |   170 |   180 |   190 |   200 |   210 |   220 |   230 |   240 |   250 |   260 |   270 |   280 |   290 |   300 |
|------------:|-----:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|        0.03 |    3 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |
|        0.05 |    3 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |
|        0.07 |    3 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |     0 |     3 |
|        0